Setting up custom chrome
#https://www.youtube.com/watch?v=Rrkj4tdXngY 

# Run the Cells Below

In [7]:
from BrowserController import BrowserControl
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import os,sys
import pandas as pd

In [8]:
browser = BrowserControl()
driver = browser.get_driver()
print("Browser OK")

	Starting custom browser
Running command: "C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe" --remote-debugging-port=9222 --user-data-dir="f:\Projects\Anom Web Scrapper\browser data" --user-agent="Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:110.0) Gecko/20100101 Firefox/110.0" --incognito
Waiting for browser to start the extension
---------------------------------------------------
Enterning sleep mode for 5 seconds
Time elapsed : 0 | Time remaining 5
Time elapsed : 1 | Time remaining 4
Time elapsed : 2 | Time remaining 3
Time elapsed : 3 | Time remaining 2
Time elapsed : 4 | Time remaining 1
---------------------------------------------------

	Setting up browser
Setting port - 9222
Setting user agent - Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:110.0) Gecko/20100101 Firefox/110.0
Creating session in incognito mode

	Browser Capabiliteies:
browserName : chrome
pageLoadStrategy : normal
goog:chromeOptions : {'debuggerAddress': 'localhost:9222', 'extensions': [], 'args': ['

# Amazon Review Parsing

In [9]:
def get_curr_page_soup():
    html = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(html, features="html.parser")
    return soup

def prod_name_link_gen(soup): # nav to list of product then run this
    # all_products = soup.select(".s-line-clamp-4")
    all_products = soup.select(".a-spacing-small.s-padding-right-small") # selects whole box with name,price stuff
    for prod in all_products:
        prod_name = prod.text
        prod_link = "https://www.amazon.com/" + prod.a["href"]
        prod_num_reviews = prod.a.parent.text
        yield prod_name, prod_link

def checkelementexist(single_elem):
    try:
        driver.find_element(by=By.CSS_SELECTOR, value=single_elem)
        return True
    except:
        return False

def goto_review_page(): # do this after selecting a product
    if checkelementexist("#reviews-medley-footer .a-text-bold"):
        elem_hlink_more = driver.find_element(by=By.CSS_SELECTOR, value="#reviews-medley-footer .a-text-bold")
        elem_hlink_more.click()
    else:
        print(r'"View All Review" not found')

def next_review():
    if checkelementexist(".a-pagination .a-last a"):
        next_btn = driver.find_element(by=By.CSS_SELECTOR, value=".a-pagination .a-last a")
        next_btn.click()
        browser.halt(3)
        return True
    else:
        return False

def translatetoEnglish(): # if available
    if checkelementexist(".cr-translate-these-reviews-section .a-link-normal"):
        print("Looking For Translation")
        browser.halt(3)
        next_btn = driver.find_element(by=By.CSS_SELECTOR, value=".cr-translate-these-reviews-section .a-link-normal")
        next_btn.click()
        print("Traslated to English")
        browser.halt(5)

def parse_review_page():
    # rating
    review_ratings = driver.execute_script('''
    content = document.querySelectorAll("#cm_cr-review_list .a-section .review-rating");
    var content_list = [];
    for(var i = 0; i < content.length; i++) {
    	content_list.push(content[i].textContent);
    }
    return content_list;
    ''')

    # review title
    review_titles = driver.execute_script('''
    content = document.querySelectorAll("#cm_cr-review_list .a-section .a-text-bold span");
    var content_list = [];
    for(var i = 0; i < content.length; i++) {
	    content_list.push(content[i].textContent);
    }
    return content_list;
    ''')

    # review content
    review_contents = driver.execute_script('''
    content = document.querySelectorAll(".a-spacing-small.review-data");
    var content_list = [];
    for(var i = 0; i < content.length; i++) {
	    content_list.push(content[i].textContent);
    }
    return content_list;
    ''')

    review_date = driver.execute_script('''
    content = document.querySelectorAll("#cm_cr-review_list .review-date");
    var content_list = [];
    for(var i = 0; i < content.length; i++) {
	    content_list.push(content[i].textContent);
    }
    return content_list;
    ''')    
    
    return review_ratings, review_titles, review_contents, review_date

def datatodataframe2(review_ratings,review_titles,review_contents,review_date):
    df = pd.DataFrame(list(zip(review_ratings, review_titles,review_contents,review_date)),columns =['Ratings', 'Title',"Content","Date"])
    return df    

def appendtocsv2(file,df):
    if os.path.exists(file):
        df.to_csv(file, mode='a', header=False)
    else:
        df = pd.DataFrame(columns =['Ratings', 'Title',"Content","Date"])
        df.to_csv(file)

In [10]:
if __name__ == "__main__":
    
    driver.get(r"https://www.amazon.com/")

In [12]:

# WTIH DATE
i = 0
while(True):
    # translatetoEnglish() #  messes up the line up
    # if checkelementexist(".a-alert-heading"): # there was a problem in review
    #     print("Review Load Error")
    #     driver.refresh()
    review_ratings,review_titles,review_contents,review_date = parse_review_page()
    if len(review_titles) > 10:
        print("Titles out of bounds")
    if len(review_contents) > 10:
        print("Review data out of bounds")
    if len(review_ratings) > 10:
        print("Rating out of bounds")
    if len(review_date) > 10:
        print("Date out of bounds")

    df = datatodataframe2(review_ratings,review_titles,review_contents,review_date)
    appendtocsv2(r"data/DS4_Test-0.txt",df)
    i = i+1
    print("--------Index {}----------".format(i))
    if next_review() == False:
        break

    

--------Index 1----------
---------------------------------------------------
Enterning sleep mode for 3 seconds
Time elapsed : 0 | Time remaining 3
Time elapsed : 1 | Time remaining 2
Time elapsed : 2 | Time remaining 1
---------------------------------------------------
Titles out of bounds
--------Index 2----------
---------------------------------------------------
Enterning sleep mode for 3 seconds
Time elapsed : 0 | Time remaining 3
Time elapsed : 1 | Time remaining 2
Time elapsed : 2 | Time remaining 1
---------------------------------------------------
Titles out of bounds
--------Index 3----------
---------------------------------------------------
Enterning sleep mode for 3 seconds
Time elapsed : 0 | Time remaining 3
Time elapsed : 1 | Time remaining 2
Time elapsed : 2 | Time remaining 1
---------------------------------------------------
Titles out of bounds
--------Index 4----------
---------------------------------------------------
Enterning sleep mode for 3 seconds
Time 

In [5]:
import pandas as pd
df = pd.read_csv("data/DS4_Test-0 copy.txt")

In [4]:
df.head(100)

,Unnamed: 0,Ratings,Title,Content,Date
0,0,5.0 out of 5 stars,Finally! A Genuine Dualshock 4,\n\n\n\n\n\n\n\n \n \n Pro-Tip: If the se...,"Reviewed in the United States on September 30,..."
1,1,4.0 out of 5 stars,Seems to have less sensitivity than the OG PS4...,\n\n\n\n\n\n\n\n \n \n This one takes a f...,"Reviewed in the United States on October 12, 2022"
2,2,5.0 out of 5 stars,Grandkids love it!,\n\n\n\n\n\n\n\n \n \n Love the color sel...,"Reviewed in the United States on September 29,..."
3,3,5.0 out of 5 stars,You can really feel that PlayStation quality,\n\n\n\n\n\n\n\n \n \n About what you'd e...,"Reviewed in the United States on September 21,..."
4,4,5.0 out of 5 stars,Great controller,\n\n\n\n\n\n\n\n \n \n This is a great co...,"Reviewed in the United States on September 27,..."
...,...,...,...,...,...
95,5,3.0 out of 5 stars,"Solid controller, until its not.",\n\n\n\n\n\n\n\n \n \n Like most people w...,"Reviewed in the United States on September 3, ..."
96,6,4.0 out of 5 stars,"Came in and works great, but is very expensive",\n\n\n\n\n\n\n\n \n \n As I am currently ...,"Reviewed in the United States on March 16, 2022"
97,7,5.0 out of 5 stars,Always Reliable,\n\n\n\n\n\n\n\n \n \n Normal ps4 control...,"Reviewed in the United States on August 12, 2022"
98,8,5.0 out of 5 stars,Great controller,\n\n\n\n\n\n\n\n \n \n Came quick and on ...,"Reviewed in the United States on July 19, 2022"


# Make your own parser bot here

In [ ]:
if __name__ == "__main__":
    
    driver.get(r"https://yoursite.com/")

In [ ]:
date = driver.execute_script('''
    // JS code goes here
    return data; // must return the data you collected using js
    ''')    

In [ ]:
date